In [45]:
import pandas as pd
# Load dataset
df = pd.read_csv('../Data/medical_no_show.csv')
#date time manage
df['ScheduledDay'] = pd.to_datetime(df['ScheduledDay'])
df['AppointmentDay'] = pd.to_datetime(df['AppointmentDay'])
df['No_show'] = (df['No-show'] == 'Yes').astype(int)
# Drop unused columns
df.drop(['No-show','PatientId','AppointmentID'], axis=1, inplace=True)
# waiting days
df['WaitingDays'] = (df['AppointmentDay'].dt.normalize() - df['ScheduledDay'].dt.normalize()).dt.days
df = df[(df['WaitingDays'] >= 0) & (df['Age'] >= 0)]

print(df['No_show'].value_counts(normalize=True)) # imbalanced data

No_show
0    0.798102
1    0.201898
Name: proportion, dtype: float64


### As we can see the data is imabalanced

## Preprocessing and Feature Engineering

In [46]:
# Encode categorical and binary features
df['Gender'] = (df['Gender']=='M').astype(int)
for col in ['Scholarship','Hipertension','Diabetes','Alcoholism','Handcap','SMS_received']:
    df[col] = df[col].astype(int)
# Extract and one-hot encode appointment weekday
df['Weekday'] = df['AppointmentDay'].dt.weekday
weekday_dummies = pd.get_dummies(df['Weekday'], prefix='WD', drop_first=True)
df = pd.concat([df, weekday_dummies], axis=1)
# Drop date cols
df.drop(['ScheduledDay','AppointmentDay','Weekday'], axis=1, inplace=True)
# Label encode neighbourhood
df['Neighbourhood'] = df['Neighbourhood'].astype('category').cat.codes

## Train-Test Split, Scaling, and Class Imbalance Handling

In [47]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import torch
# Features and target
X = df.drop('No_show', axis=1)
y = df['No_show'].values
# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)
# Scale features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)
# Compute positive class weight
num_pos = (y_train==1).sum()
num_neg = (y_train==0).sum()
pos_weight = torch.tensor(num_neg/num_pos, dtype=torch.float32)

In [48]:
X_train.shape[1]

15

### Timer decorator

In [49]:
import time
def timer(func):
    def wrapper(*args, **kwargs):
        start = time.perf_counter()
        result = func(*args, **kwargs)
        end = time.perf_counter()
        duration = end - start
        if args and hasattr(args[0], '__dict__'):
            setattr(args[0], f'{func.__name__}_time', duration)
        print(f"Function '{func.__name__}' took {duration:.4f} seconds")
        return result
    return wrapper

## Model Definition with Weighted Loss

In [50]:
import torch.nn as nn
import torch.optim as optim
torch.manual_seed(42)
input_dim = X_train.shape[1]
class NoShowNet(nn.Module):
    def __init__(self, dim):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(dim,64),
            nn.BatchNorm1d(64), nn.ReLU(), nn.Dropout(0.5),
            nn.Linear(64,32), nn.BatchNorm1d(32), nn.ReLU(), nn.Dropout(0.5),
            nn.Linear(32,1)
        ) # use of LLMs for finding out these different types of layer sructure
    def forward(self, x):
        return self.net(x).squeeze(1)
model = NoShowNet(input_dim)
criterion = nn.BCEWithLogitsLoss(pos_weight=pos_weight)
optimizer = optim.Adam(model.parameters(), lr=1e-3)
model

NoShowNet(
  (net): Sequential(
    (0): Linear(in_features=15, out_features=64, bias=True)
    (1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Dropout(p=0.5, inplace=False)
    (4): Linear(in_features=64, out_features=32, bias=True)
    (5): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): ReLU()
    (7): Dropout(p=0.5, inplace=False)
    (8): Linear(in_features=32, out_features=1, bias=True)
  )
)

## Preparing DataLoader

In [51]:
from torch.utils.data import TensorDataset, DataLoader
# Convert to tensors
X_tr = torch.tensor(X_train, dtype=torch.float32)
y_tr = torch.tensor(y_train, dtype=torch.float32)
X_te = torch.tensor(X_test, dtype=torch.float32)
y_te = torch.tensor(y_test, dtype=torch.float32)
# DataLoader
dataset = TensorDataset(X_tr, y_tr)
loader = DataLoader(dataset, batch_size=256, shuffle=True)

## Training Loop

In [52]:
epochs = 50

@timer
def train_loop():
    for epoch in range(epochs):
        model.train()
        total_loss = 0.0
        for xb, yb in loader:
            optimizer.zero_grad()
            out = model(xb)
            loss = criterion(out, yb)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()*xb.size(0)
        print(f'Epoch {epoch+1}/{epochs} - Loss: {total_loss/len(dataset):.4f}')

train_loop()

Epoch 1/50 - Loss: 1.0741
Epoch 2/50 - Loss: 1.0400
Epoch 3/50 - Loss: 1.0247
Epoch 4/50 - Loss: 1.0160
Epoch 5/50 - Loss: 1.0111
Epoch 6/50 - Loss: 1.0030
Epoch 7/50 - Loss: 0.9993
Epoch 8/50 - Loss: 0.9933
Epoch 9/50 - Loss: 0.9906
Epoch 10/50 - Loss: 0.9855
Epoch 11/50 - Loss: 0.9838
Epoch 12/50 - Loss: 0.9845
Epoch 13/50 - Loss: 0.9833
Epoch 14/50 - Loss: 0.9802
Epoch 15/50 - Loss: 0.9782
Epoch 16/50 - Loss: 0.9784
Epoch 17/50 - Loss: 0.9784
Epoch 18/50 - Loss: 0.9771
Epoch 19/50 - Loss: 0.9774
Epoch 20/50 - Loss: 0.9776
Epoch 21/50 - Loss: 0.9770
Epoch 22/50 - Loss: 0.9747
Epoch 23/50 - Loss: 0.9765
Epoch 24/50 - Loss: 0.9752
Epoch 25/50 - Loss: 0.9741
Epoch 26/50 - Loss: 0.9777
Epoch 27/50 - Loss: 0.9739
Epoch 28/50 - Loss: 0.9748
Epoch 29/50 - Loss: 0.9751
Epoch 30/50 - Loss: 0.9749
Epoch 31/50 - Loss: 0.9738
Epoch 32/50 - Loss: 0.9723
Epoch 33/50 - Loss: 0.9743
Epoch 34/50 - Loss: 0.9754
Epoch 35/50 - Loss: 0.9721
Epoch 36/50 - Loss: 0.9733
Epoch 37/50 - Loss: 0.9736
Epoch 38/5

## Evaluation with Dynamic Threshold Selection

In [53]:
from sklearn.metrics import precision_recall_curve, accuracy_score, precision_score, recall_score, f1_score, average_precision_score, confusion_matrix
import numpy as np
model.eval()
with torch.no_grad():
    logits = model(X_te)
    probs = torch.sigmoid(logits).numpy()
prec, rec, thr = precision_recall_curve(y_test, probs)
f1s = 2*(prec*rec)/(prec+rec+1e-8)
best = np.argmax(f1s)
best_thr = thr[best] if best < len(thr) else 0.5
print(f'Best threshold: {best_thr:.3f}, F1: {f1s[best]:.3f}')
preds = (probs>=best_thr).astype(int)
print('Accuracy:', accuracy_score(y_test, preds))
print('Precision:', precision_score(y_test, preds))
print('Recall:', recall_score(y_test, preds))
print('F1 Score:', f1_score(y_test, preds))
print('PR AUC:', average_precision_score(y_test, probs))

print('Confusion matrix: ', confusion_matrix(y_test, preds))

Best threshold: 0.513, F1: 0.443
Accuracy: 0.5919022845510066
Precision: 0.30564557393825686
Recall: 0.8030472776159534
F1 Score: 0.44276978195070726
PR AUC: 0.35595558885665346
Confusion matrix:  [[9500 8142]
 [ 879 3584]]
